<a href="https://colab.research.google.com/github/AlexeyTri/Mathematics/blob/main/%D0%9B%D0%B8%D0%BD%D0%B5%D0%B9%D0%BD%D0%B0%D1%8F_%D1%80%D0%B5%D0%B3%D1%80%D0%B5%D1%81%D1%81%D0%B8%D1%8F.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**Линейная регрессия (linear regression)** - алгоритм машинного обучения, описывающий зависимость целевой переменной от признака в виде линейной функции:

$y = kx + b$

$f_{w, b}(x) = w_0x_0 + w_1x_1 + ... + w_nx_n + b$

w - weigths, вектор весов $\in ℝ^{1 \times n}$. Совокупность весов всех экземпляров образуют матрицу весов $W \in ℝ^{n \times k}$

k - target, количество целевых классов

n - количество признаков

x - features, вектор признаков одного обучающего экземпляра $\in ℝ^{1 \times n}$. Совокупность признаков всех экземпляров образуют матрицу признакового описания $X \in ℝ^{l \times n}$

l - количество экземпляров

b - bias, отступ модели

**Функция потерь** описывает разницу между значением целевой переменной (значением класса) и результатом предсказания алгоритма.
Согласно теоремы Гаусса-Маркова, для алгоритма линейной регрессии наиболее оптимальной является функция среднеквадратичной ошибки:

$𝓛 = ||\hat y - y||^2$, где

$\hat y$ - результат работ алгоритма $\in ℝ$, в задачах линейно регресии
$y$ - целевая переменная


В векторной/матричной форме:

$𝓛(w, X, y) = \frac{1}{2l}∑_{i=1}^{l}(w^Tx_i-y_i)^2 = \\
\frac{1}{2l}||Xw - y||^2 = \frac{1}{2l}(Xw - y)^T(Xw - y) =\\
\frac{1}{2l}((Xw)^T(Xw) - (Xw)^Ty - y^T(Xw) + y^Ty) ⇒\\
\frac{𝑑𝓛}{𝑑w}=0 ⇒ \frac{1}{2l}(2X^TXw - 2X^Ty) = 0 ⇒\\
X^TXw = X^Ty ⇒\\
w = (X^TX)^{-1}X^Ty$

*Способы вычисления $w$:*

1. на основании аналитической формулы, выведенной выше. Основные проблемы с реализацией в больших вычислительных затратах для расчета обратной матрицы
2. итеративный способ, на основании расчета градиетна и последующего шага функции потерь в данном направлении:

$\frac{𝑑𝓛}{𝑑w}=0 ⇒ \frac{1}{l}∑_{i=1}^{l}(f_{w,b}(x_i) - y_i)x_i\\
\frac{𝑑𝓛}{𝑑b}=0 ⇒ \frac{1}{l}∑_{i=1}^{l}(f_{w,b}(x_i) - y_i)\\
b_j = b_{j-1} - α * \frac{𝑑𝓛}{𝑑b_j}\\
w_j = w_{j-1} - α * \frac{𝑑𝓛}{𝑑w_j}$










# Задача: спрогнозировать доход сотрудников на основании их опыта и возраста

In [1]:
# dataset URL: https://www.kaggle.com/datasets/hussainnasirkhan/multiple-linear-regression-dataset

import pandas as pd
import requests
import io
import zipfile

zipped = zipfile.ZipFile("/content/drive/MyDrive/Colab Notebooks/archive.zip")
zfile = zipped.open(zipped.infolist()[0])
df=pd.read_csv(zfile)

In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.datasets import make_regression
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import scale, PolynomialFeatures
from sklearn.linear_model import LinearRegression, Ridge, Lasso, ElasticNet
from sklearn.metrics import mean_absolute_percentage_error, mean_squared_error, r2_score

In [16]:
class MatrixLinearRegression:

    def fit(self, X, y):
        X = np.insert(X, 0, 1, axis=1) #add vector b, l - dimmension
        X_inv_X = np.linalg.inv(np.dot(X.T,X)) # (X.T * X) ** -1
        weights = np.linalg.multi_dot([X_inv_X, X.T, y])
        self.bias, self.weights = weights[0], weights[1:]

    def predict(self, X_test):
        return X_test @ self.weights + self.bias

In [19]:
class GDLinearRegression:
    def __init__(self, learning_rate=0.01, tolerance=1e-8):
        self.learning_rate = learning_rate
        self.tolerance = tolerance

    def fit(self, X, y):
        n_samples, n_features = X.shape
        self.bias, self.weights = 0, np.zeros(n_features)
        previos_db, previos_dw = 0, np.zeros(n_features)

        while True:
            y_pred = X @ self.weights + self.bias
            db = 1/ n_samples * np.sum(y_pred - y)
            dw = 1/ n_samples * X.T @ (y_pred - y)
            self.bias -= self.learning_rate * db
            self.weights -= self.learning_rate * dw

            abs_db_reduction = np.abs(db - previos_db)
            abs_dw_reduction = np.abs(dw - previos_dw)

            if abs_db_reduction < self.tolerance:
                if abs_dw_reduction.all() < self.tolerance:
                    break

            previos_db = db
            previos_dw = dw

    def predict(self, X_test):
        return X_test @ self.weights + self.bias

In [6]:
X1, y1 = df.iloc[:, :-1].values, df.iloc[:, -1].values
X1_scaled = scale(X1)
X1_train, X1_test, y1_train, y1_test = train_test_split(X1, y1, random_state=0)
X1_train_s, X1_test_s, y1_train, y1_test = train_test_split(X1_scaled, y1, random_state=0)

correlation_matrix = df.corr()
correlation_matrix.style.background_gradient(cmap='coolwarm')

,age,experience,income
age,1.000000,0.615165,0.532204
experience,0.615165,1.000000,0.984227
income,0.532204,0.984227,1.000000


In [17]:
matrix_linear_regression = MatrixLinearRegression()
matrix_linear_regression.fit(X1_train_s, y1_train)
matrix_lr_pred_res = matrix_linear_regression.predict(X1_test_s)
matrix_lr_r2 = r2_score(y1_test, matrix_lr_pred_res)
matrix_lr_mape = mean_absolute_percentage_error(y1_test, matrix_lr_pred_res)

print(f'Matrix Linear regression  R2 score: {matrix_lr_r2}')
print(f'Matrix Linear regression MAPE: {matrix_lr_mape}', '\n')

print(f'weights: {matrix_linear_regression.bias, *matrix_linear_regression.weights}')
print(f'prediction: {matrix_lr_pred_res}')

Matrix Linear regression  R2 score: 0.9307237996010834
Matrix Linear regression MAPE: 0.04666577176525877 

weights: (np.float64(40922.38666080836), np.float64(-1049.7866043343445), np.float64(8718.76435636617))
prediction: [46528.00800666 35018.47848628 49448.73803373 38604.36954966
 30788.13913983]


In [20]:
linear_regression = GDLinearRegression()
linear_regression.fit(X1_train_s, y1_train)
pred_res = linear_regression.predict(X1_test_s)
r2 = r2_score(y1_test, pred_res)
mape = mean_absolute_percentage_error(y1_test, pred_res)

print(f'Linear regression R2 score: {r2}')
print(f'Linear regression MAPE: {mape}', '\n')

print(f'weights: {linear_regression.bias, *linear_regression.weights}')
print(f'prediction: {pred_res}')

Linear regression R2 score: 0.9307237996010985
Linear regression MAPE: 0.04666577176525461 

weights: (np.float64(40922.386660807955), np.float64(-1049.7866043338142), np.float64(8718.76435636563))
prediction: [46528.00800666 35018.47848628 49448.73803373 38604.36954966
 30788.13913983]


In [21]:
sk_linear_regression = LinearRegression()
sk_linear_regression.fit(X1_train, y1_train)

sk_lr_pred_res = sk_linear_regression.predict(X1_test)
sk_lr_r2 = r2_score(y1_test, sk_lr_pred_res)
sk_lr_mape = mean_absolute_percentage_error(y1_test, sk_lr_pred_res)

print(f'Scikit-learn Linear regression R2 score: {sk_lr_r2}')
print(f'Scikit-learn Linear regression MAPE: {sk_lr_mape}', '\n')

print(f'weights: {sk_linear_regression.intercept_, *sk_linear_regression.coef_}')
print(f'prediction: {sk_lr_pred_res}', '\n')

sk_linear_regression.fit(X1_train_s, y1_train)
print(f'scaled weights: {sk_linear_regression.intercept_, *sk_linear_regression.coef_}')

Scikit-learn Linear regression R2 score: 0.9307237996010832
Scikit-learn Linear regression MAPE: 0.046665771765258775 

weights: (np.float64(31734.098811233787), np.float64(-107.40804717984585), np.float64(2168.8736968153985))
prediction: [46528.00800666 35018.47848628 49448.73803373 38604.36954966
 30788.13913983] 

scaled weights: (np.float64(40922.38666080837), np.float64(-1049.7866043343417), np.float64(8718.764356366162))


**Дмашнее задание 4.1:**



1.   реализовать функцию scale - 25 баллов
2.   реализовать функцию R2_Score - 25 баллов
3.   реализовать функцию MAPE - 25 баллов
4.   реализовать пайплайн ЛинейнойРегресии с полученными функциями - 25 баллов

